In [49]:
from utils import load_file
import numpy as np

In [50]:
res_path = r"eval_res\selfrag-pqa-fullspan.json"
data_path = r"eval_data\popqa_longtail_w_gs.jsonl"

In [51]:
data = load_file(data_path)
res = load_file(res_path)

In [52]:
data[0].keys()

dict_keys(['question', 'answers', 'prop', 's_wiki_title', 'id', 'pop', 'ctxs'])

In [53]:
res['results'][0].keys()

dict_keys(['no_retrieval', 'no_retrieval_ids', 'no_retrieval_log_probs', 'all_doc_retrieval', 'all_doc_retrieval_ids', 'all_doc_retrieval_log_probs', 'retrieval', 'retrieval_token_ids', 'retrieval_log_probs', 'retrieval_p', 'retrieval_res', 'question_id', 'gold'])

In [54]:
res['results'][0]['retrieval_res']['retrieval_0'].keys()

dict_keys(['pred', 'score', 'id_log_probs', 'token_ids'])

In [55]:
def false_invperplexity(ret_res):
    return np.sum(ret_res['id_log_probs']) / max(1, len(ret_res['token_ids']))

def true_invperplexity(ret_res):
    return np.sum(ret_res['id_log_probs']) / max(1, len(ret_res['id_log_probs']))

In [56]:
def recompute_score(ret_res):
    res = ret_res.copy()
    ret_prefix = ['retrieval_0', 'retrieval_1', 'retrieval_2', 'retrieval_3', 'retrieval_4']
    for prefix in ret_prefix:
        global res_path
        assert len(res_path) > 0
        if 'selfrag' in res_path:
            assert false_invperplexity(ret_res[prefix]) != true_invperplexity(ret_res[prefix])
            score = ret_res[prefix]['score'] - false_invperplexity(ret_res[prefix]) + true_invperplexity(ret_res[prefix])
            
        else:
            score = true_invperplexity(ret_res[prefix])
        res[prefix]['score'] = score
    return res
        

In [57]:
for i in res['results']:
    i['retrieval_res'] = recompute_score(i['retrieval_res'])
    if 'arc' in res_path or 'health' in res_path:
        raise NotImplementedError
    else:
        path2score = {key: item["score"] for key,
                        item in i['retrieval_res'].items()}
        best_path = sorted(path2score.items(),
                            key=lambda x: x[1], reverse=True)[0][0]
        best_option = i['retrieval_res'][best_path]["pred"]
        token_ids = i['retrieval_res'][best_path]["token_ids"]
        id_log_probs =i['retrieval_res'][best_path]["id_log_probs"]
        i["best_one"] = i['retrieval_res'][best_path]
    
    i["retrieval"] = best_option
    i["retrieval_token_ids"] = token_ids
    i["retrieval_log_probs"] = id_log_probs
        

In [58]:

import json
with open(res_path.replace(".json", "_recomputed.json"), 'w') as f:
    json.dump(res, f)